In [ ]:
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
from protocols import nogse
import os
import seaborn as sns

# Configuración de estilo
def set_style():
    sns.set_theme(context='paper')
    sns.set_style("whitegrid")
set_style()

# Widgets para la entrada de datos
file_name = widgets.Text(value="tesis_final_final/results_levaduras_20240613", description="File Name:")
data_directory = widgets.Text(value="C:/Users/Ignacio Lembo/data/data_levaduras_20240613", description="Data Dir:")
folder = widgets.Text(value="contrast_vs_g_data", description="Folder:")
rois = widgets.Text(value="ROI1", description="ROIs (comma sep):")
masks = widgets.Text(value="1", description="Masks (comma sep):")
slic = widgets.IntText(value=0, description="Slice:")
run_button = widgets.Button(description="Run Analysis")

# Mostrar widgets
display(file_name, data_directory, folder, rois, masks, slic, run_button)

def run_analysis(b):
    set_style()
    rois_list = rois.value.split(',')
    masks_list = list(map(int, masks.value.split(',')))
    
    palette = sns.color_palette("tab10", len(rois_list) + 1)
    
    image_paths_A0, method_paths_A0 = nogse.upload_contrast_data_A0_v2(data_directory.value, 0)
    image_paths, method_paths = nogse.upload_contrast_data_v2(data_directory.value, 0)
    
    mask = np.loadtxt(f"rois/mask_2.txt")
    tnogse, g_contrast, n, f_A0_hahn_noise, f_A0_cpmg_noise = nogse.generate_contrast_vs_g_roi(image_paths_A0, method_paths_A0, mask, slic.value)
    tnogse, g_contrast, n, f_hahn_noise, f_cpmg_noise = nogse.generate_contrast_vs_g_roi(image_paths, method_paths, mask, slic.value)
    
    f_noise = (np.array(f_cpmg_noise) + np.array(f_hahn_noise)) / 2
    fig, ax = plt.subplots(figsize=(8, 6))
    
    for roi, mask, color in zip(rois_list, masks_list, palette):
        mask = np.loadtxt(f"rois/mask_{mask}.txt")
        fig1, ax1 = plt.subplots(figsize=(8, 6))
        
        f_A0_cpmg_noise = np.zeros(len(g_contrast))
        f_A0_hahn_noise = np.zeros(len(g_contrast))
        f_cpmg_noise = np.zeros(len(g_contrast))
        f_hahn_noise = np.zeros(len(g_contrast))
        
        tnogse, g, n, f_A0_hahn, f_A0_cpmg = nogse.generate_contrast_vs_g_roi_A0_riciannoise(image_paths_A0, method_paths_A0, mask, slic.value, f_A0_hahn_noise, f_A0_cpmg_noise)
        tnogse, g, n, f_hahn, f_cpmg = nogse.generate_contrast_vs_g_roi_riciannoise(image_paths, method_paths, mask, slic.value, f_hahn_noise, f_cpmg_noise)
        
        f = f_cpmg / f_A0_cpmg - f_hahn / f_A0_hahn
        directory = f"../results_{file_name.value}/{folder.value}/tnogse={tnogse}_N={int(n)}"
        os.makedirs(directory, exist_ok=True)
        
        nogse.plot_contrast_vs_g_data(ax, roi, g, f, np.abs((np.std(f_noise) / np.mean(f_noise)) * np.array(f)), tnogse, n, slic.value, color)
        nogse.plot_contrast_vs_g_data(ax1, roi, g, f, np.abs((np.std(f_noise) / np.mean(f_noise)) * np.array(f)), tnogse, n, slic.value, color)
        
        table = np.vstack((g, f, (np.std(f_noise) / np.mean(f_noise)) * np.array(f), f_cpmg, (np.std(f_noise) / np.mean(f_noise)) * np.array(f_cpmg), f_hahn, (np.std(f_noise) / np.mean(f_noise)) * np.array(f_hahn)))
        np.savetxt(f"{directory}/{roi}_data_contrast_vs_g_tnogse={tnogse}_N={int(n)}.txt", table.T, delimiter=' ', newline='\n')
        
        fig1.tight_layout()
        fig1.savefig(f"{directory}/{roi}_contrast_vs_g_tnogse={tnogse}_N={n}.pdf")
        fig1.savefig(f"{directory}/{roi}_contrast_vs_g_tnogse={tnogse}_N={n}.png", dpi=600)
        plt.close(fig1)
    
    fig.tight_layout()
    fig.savefig(f"{directory}/contrast_vs_g_tnogse={tnogse}_N={n}.pdf")
    fig.savefig(f"{directory}/contrast_vs_g_tnogse={tnogse}_N={n}.png", dpi=600)
    plt.close(fig)
    print("Analysis completed!")

run_button.on_click(run_analysis)
import ipywidgets as widgets
from IPython.display import display, Javascript

def toggle_code(_):
    display(Javascript('''
        code_show = !code_show;
        code_input = document.querySelectorAll('.input');
        for (var i = 0; i < code_input.length; i++) {
            code_input[i].style.display = code_show ? 'none' : 'block';
        }
    '''))

toggle_button = widgets.Button(description="Mostrar/Ocultar Código")
toggle_button.on_click(toggle_code)
display(toggle_button)


Text(value='tesis_final_final/results_levaduras_20240613', description='File Name:')

Text(value='C:/Users/Ignacio Lembo/data/data_levaduras_20240613', description='Data Dir:')

Text(value='contrast_vs_g_data', description='Folder:')

Text(value='ROI1', description='ROIs (comma sep):')

Text(value='1', description='Masks (comma sep):')

IntText(value=0, description='Slice:')

Button(description='Run Analysis', style=ButtonStyle())

In [ ]:
from IPython.core.display import HTML

HTML('''
<script>
code_show = false; 
function code_toggle() {
    if (code_show){
        $('div.input').show();
    } else {
        $('div.input').hide();
    }
    code_show = !code_show
} 
$(document).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Mostrar/Ocultar Código"></form>
''')